# Modèle 1 : Architecture comparaison

## Définition d'une procédure de test

**Dataset :**

Afin de pouvoir comparer nos architectures et par la suite nos modèles, nous avons généré un dataset d'entrainement.
Nous avons utilisé Chat-GPT en lui donnant des Id_solution ainsi que toutes les informations que nous utilisons dans notre modèle (Titre, Description, Bilan énergie) afin qu'il génère des requêtes qu'un utilisateur aurrait pu formuler en fonction de ces solutions. Nous avons tiré les id_solution de manière aléatoire afin d'avoir une répartition des question homogène sur le dataset. Nous n'avons pas de doublons d'id_solution dans le dataset.

**Méthode de test :**

Afin de tester l'accuracy de nos modèles, nous avons choisi de faire des inférence sur notre dataset de test et de compter 1 point si l'id_solution réel est prédit en première position et 0 point sinon. Ensuite on divise par le nombre de données dans le dataset afin d'avoir une valeur d'accuracy entre 0 et 1.

Il est evident que notre modèle prédira en 2, 3, ... positions d'autres résultats pertinents que l'utilisateur pourrait étudier, mais cela n'étant pas évident à quantifier dans un premier test automatique, nous avons choisi de ne pas le traiter pour le moment.

## Tests

Les codes de ces trois architectures utilisant des fonctions similaires mais avec quelques subtiles adaptations, nous avons préféré lancer les tests dans chacun des fichiers .ipynb de nos architecture et recensé les résultats ici.

## Imports

In [1]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np
import random

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Anciennes fonctions

### Commun

In [2]:
def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

### Architecture 1

### Architecture 2

### Architecture 3

In [52]:
def pre_processing_sol_archi3(sentences):
    phrases_propres = []
    for texte in sentences:
        # Nettoyer HTML Tags
        texte = BeautifulSoup(texte, 'html.parser').get_text()
        # Remplacer "&nbsp;." par rien
        texte = re.sub(r'&nbsp;\.', '', texte)
        # Accents
        texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")
        # Retirer les numéros
        texte = re.sub(r'\b\d+\b', '', texte)
        # Tokenization, Lemmatization, Removing Stopwords, Lowercase
        doc = nlp(texte)
        for phrase in doc.sents:
            tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
            phrase_propre = ' '.join(tokens)
            if phrase_propre:
                phrases_propres.append(phrase_propre)
    return phrases_propres

def load_and_merge_data_archi3(csv_file = '../data/solutions.csv'):
    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)
    # Renommer les colonnes
    df.columns = ['id_solution', 'categorie', 'texte']
    # Filtrer les lignes pour les catégories spécifiées
    categories_specifiees = [1, 2, 5, 6]
    df_filtre = df[df['categorie'].isin(categories_specifiees)]
    # Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
    df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()
    # Renommer les colonnes
    df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']
    # Gérer les valeurs NaN lors de la fusion des colonnes
    def combine_text(row):
        text_parts = [row[col] for col in colonnes if pd.notnull(row[col])]
        return text_parts
    # Sélectionner uniquement les colonnes 'id_solution' et les champs requis
    colonnes = ['titre', 'definition', 'application', 'bilan_energie']
    df_pivot['champs'] = df_pivot.apply(combine_text, axis=1)
    df_final = df_pivot[['id_solution', 'champs']]
    # Convertir en liste de listes pour chaque ligne
    result = df_final.values.tolist()
    return result

# NOUVELLE FONCTION
def calculate_section_embedding_archi3(sentences, quantize=False, precision="binary"):
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    return sentence_embeddings

def genere_embedding_archi3(data, output_file, quantize=False, precision="binary"):
    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_section_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_section_embedding)
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
    }
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    print("Les embeddings ont été storer avec succès.")


def find_solution_archi3(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)
    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)
        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1), embeddings_array)
        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)
    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    return solution_list

# Fonction appelé par notre utilisateur
def model_PAT_archi3(secteur, description, embedding_name) :
    # On commence par concaténer notre secteur et notre description.
    text = secteur + ". " + description
    # Ensuite on applique notre pré-processing
    clean_text = pre_processing(text)
    # Ensuite on cherche nos similarités 
    solutions = find_solution_archi3(clean_text, embedding_name)
    # On return une liste contenant uniquement le numéros des solutions
    id_solutions = []
    for solution in solutions :
        id_solutions.append(solution[0])
    return id_solutions

df_solution_archi3 = load_and_merge_data_archi3()

## Récupération de quelques données aléatoire pour générer notre dataset dans ChatGPT

In [44]:
# Chargement et fusion des données (simulation)
df_solutions = load_and_merge_data_archi3()

# Générer et afficher 5 identifiants de solution aléatoires
for _ in range(10):
    id_solution = random.randint(0, 1400)
    if (df_solutions[id_solution]):
        print("Id solution : ", df_solutions[id_solution][0], "\nDescription : ", df_solutions[id_solution][1])

Id solution :  249 
Description :  ['Moteur à haut rendement label IE2 (EFF1)']
Id solution :  167 
Description :  ['Participation du personnel dans la recherche de fuite', 'Le personnel qui travaille directement sur les lignes de production&nbsp;est id&eacute;alement plac&eacute; pour identifier les fuites&nbsp;en premier.&nbsp;De ce fait, l&rsquo;implication du personnel dans la recherche de fuites sur le r&eacute;seau d&rsquo;air comprim&eacute; peut aider &agrave; en optimiser le fonctionnement. Le personnel sensibilis&eacute; travaillant peut faire remonter l&rsquo;information lorsqu&rsquo;il se rend compte d&rsquo;une fuite d&rsquo;air sur le r&eacute;seau ou de la d&eacute;faillance d&rsquo;une soufflette. Des explications compl&eacute;mentaires sur la mani&egrave;re de d&eacute;tecter les fuites peuvent &ecirc;tre n&eacute;cessaires. &nbsp;', "La solution est applicable à tous le sites industriels utilisant de l'air comprimé.&nbsp;Du fait du nombre de personnel élevé, l'identif

IndexError: list index out of range

## Chargement du dataset de test

In [50]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/patrice_test.csv")

## New fonction accuracy

In [53]:
# Entrée :
# nom_model = nom du model à tester. Attention, nous devons avoir généré l'embedding dans /embeddings
# dataset = dataset sur lequel on souhaite réaliser le test 
# top_n = integer du rang à partir du quel on considère une solution valide. 1 <=> seulement si top 1, 2 <=> top 1 ou 2 , ...
def test_accuracy(embedding_name, dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_PAT_archi3("", dataset['Description'][i], embedding_name)
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
    return accuracy

## Test architecture

Afin de pouvoir comparer nos architecture nous avons choisi un modèle. Notre étude préliminaire nous a permis de déterminer que deux modèles multilingue sortaient du lot : mpnet-base-v2 et LaBSE. Pour notre étude d'architecture nous avons du fixer un modèle de manière arbitraire, ici nous avons choisi d'utiliser mpnet-base-v2.

In [19]:
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [20]:
liste_architecture = {"Architecture 1" : "paraphrase-multilingual-mpnet-base-v2_embeddings.pkl", "Architecture 2" : "Architecure-2_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl", "Architecture 3" : "Architecure-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl"}

### Calcul d'accuracy

In [54]:
test_accuracy(liste_architecture["Architecture 1"])

FileNotFoundError: [Errno 2] No such file or directory: 'paraphrase-multilingual-mpnet-base-v2_embeddings.pkl'